In [1]:
import modelscope
from transformers import AutoTokenizer,AutoModelForCausalLM
import json
import torch

/usr/local/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('./muice-dataset-train.catgirl (2).json','r') as f:
    data = json.load(f)

In [3]:
max_len = 0
for d in data:
    max_len = max(len(d['instruction']),max_len)
    max_len = max(len(d['output']),max_len)
max_len

146

In [4]:
max_length = 256

In [5]:
model = AutoModelForCausalLM.from_pretrained("./DeepSeek")
tokenizer = AutoTokenizer.from_pretrained("./DeepSeek")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [6]:
# def token(example,max_length=180):
#     instruction = example['instruction']
#     output = example['output']
#     prompt = [{"role":"user","content":instruction}]
#     prompt_token = tokenizer.apply_chat_template(prompt,tokenize=False)
#     q = tokenizer.encode(prompt_token)
#     a = tokenizer.encode(output)
#     input_ids = q+a
#     labels = a
#     attention_mask = [-100]*len(q) + [1]*len(a)
#     if len(input_ids)<=max_length:
#         padding_length = max_length-len(input_ids)
#         input_ids = [-100]*padding_length+ input_ids
#         labels = [-100]*(padding_length+len(q)) +labels
#         attention_mask = [0]*padding_length+attention_mask
#     else:
#         input_ids = input_ids[:max_length]
#         labels = labels[:max_length]
#         attention_mask = attention_mask[:max_length]
#     return {
#         "input_ids":input_ids,
#         "labels":labels,
#         "attention_mask":attention_mask
#     }
# def token(example, max_length=180):
#     instruction = example['instruction']
#     output = example['output']
#     prompt = [{"role": "user", "content": example["instruction"]}]
#     prompt_token = tokenizer.apply_chat_template(prompt, tokenize=False)
#     tokenized = tokenizer(
#         prompt_token,
#         text_target=example["output"],
#         max_length=max_length,
#         padding="max_length",
#         truncation=True,
#         return_tensors="pt",
#     )
#     input_ids = tokenized["input_ids"][0]
#     labels = tokenized["labels"][0]
#     attention_mask = tokenized["attention_mask"][0]
  
    
#     if len(input_ids) < max_length:
#         pad_len = max_length - len(input_ids)
#         input_ids = [tokenizer.pad_token_id] * pad_len + input_ids
#         attention_mask = [0] * pad_len + attention_mask
#         labels = [-100] * pad_len + labels
#     else:
#         input_ids = input_ids[:max_length]
#         attention_mask = attention_mask[:max_length]
#         labels = labels[:max_length]
  # prompt = [{"role": "user", "content": instruction}]
    # prompt_token = tokenizer.apply_chat_template(prompt, tokenize=False)
    
    # q = tokenizer.encode(prompt_token, add_special_tokens=False)
    # a = tokenizer.encode(output, add_special_tokens=False)
    
    # input_ids = q + a
    # attention_mask = [-100] * len(q) + [1] * len(a)
    # labels = [-100] * len(q) + a
def token(example, max_length=180):
    instruction = example['instruction']
    output = example['output']
    prompt = [{"role": "user", "content": example["instruction"]}]
    q = tokenizer.apply_chat_template(prompt, tokenize=False,add_generation_prompt=True)
    
    q_input_ids = tokenizer.encode(q)
    a_input_ids = tokenizer.encode(output)
    input_ids = q_input_ids+a_input_ids
    attention_mask = [1]*len(input_ids)
    labels = [-100]*len(q_input_ids)+a_input_ids
    if len(input_ids) < max_length:
        pad_len = max_length - len(input_ids)
        input_ids = [tokenizer.pad_token_id] * pad_len + input_ids
        attention_mask = [0] * pad_len + attention_mask
        labels = [-100] * pad_len + labels
    else:
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    
    # Sanity check
    assert len(input_ids) == max_length
    assert len(labels) == max_length
    assert len(attention_mask) == max_length

    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_mask
    }

data_ids = []
for d in data:
    id_dict = token(d)
    data_ids.append(id_dict)
    

In [7]:
from torch.utils.data import Dataset
class MyData(Dataset):
    def __init__(self,data):
        super().__init__()
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        data = self.data[index]
        
        return {
            "input_ids": torch.tensor(data["input_ids"], dtype=torch.long),
            "labels": torch.tensor(data["labels"], dtype=torch.long),
            "attention_mask": torch.tensor(data["attention_mask"], dtype=torch.long)
        }

In [8]:
# data_ids[:5]['input_ids']

In [9]:
from torch.utils.data import DataLoader
my_data = MyData(data_ids)
train_dataloader = DataLoader(my_data, shuffle=True, batch_size=8)


In [10]:
tokenizer.pad_token_id

151643

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-6)
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for i,batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        
        print(loss.item())

  0%|          | 1/483 [00:45<6:03:02, 45.19s/it]

7.780388832092285
